<a href="https://colab.research.google.com/github/advik-7/NLP_projects/blob/main/Faiss_flat_index_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
import urllib.request as request
from contextlib import closing

with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [2]:

import time

In [3]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00


In [4]:
import faiss

res = faiss.StandardGpuResources()

In [5]:
import tarfile

tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [6]:
import numpy as np

def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [8]:
xb = read_fvecs('/content/sift/sift_base.fvecs')
xq = read_fvecs('/content/sift/sift_query.fvecs')
xq = xq[0].reshape(1, xq.shape[1])

In [9]:
xb.shape

(1000000, 128)

In [10]:
xq.shape

(1, 128)

In [11]:
xq

array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
         18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
        101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
          1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
         40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
        116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
         10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
          0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
        116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
          9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
          2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
         18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32)

In [12]:
xb

array([[  0.,  16.,  35., ...,  25.,  23.,   1.],
       [ 14.,  35.,  19., ...,  11.,  21.,  33.],
       [  0.,   1.,   5., ...,   4.,  23.,  10.],
       ...,
       [ 30.,  12.,  12., ...,  50.,  10.,   0.],
       [  0.,   5.,  12., ...,   1.,   2.,  13.],
       [114.,  31.,   0., ...,  25.,  16.,   0.]], dtype=float32)

In [13]:
D=128

In [14]:
M=16

In [15]:
ef_search=8
ef_construction=64

In [16]:

d = xb.shape[1]
cpu_index = faiss.IndexFlatL2(d)
gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
gpu_index.add(xb)

k = 10
start_time = time.time()
distances, indices = gpu_index.search(xq, k)

end_time = time.time()
print("Indices of nearest neighbors:", indices)
print("Distances of nearest neighbors:", distances)

Indices of nearest neighbors: [[932085 934876 561813 708177 706771 695756 435345 701258 455537 872728]]
Distances of nearest neighbors: [[54229. 55091. 59531. 65260. 65697. 67010. 68247. 69844. 71441. 71861.]]


In [17]:

print(f"Time taken for retrieval: {end_time - start_time:.4f} seconds")

Time taken for retrieval: 0.0132 seconds
